# Progress Report of ani 
## 29/1/23

1. fit regression models to `ani` and `frac_diff` for a.a.

2. compare the a.a. trees from `frac_diff` and from estimates of fractional difference by `ani`.

3. repeat 2. for more data


## Setup

In [1]:
from collections import Counter
from math import log

import numpy
import plotly.express as px
import plotly.graph_objects as go
from cogent3 import load_aligned_seqs
from cogent3.app import io, translate
from cogent3.app.align import progressive_align
from cogent3.phylo import nj
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

In [2]:
#function of the relation
def cubic_function(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d

def kfreqs(seq, k):
    return Counter([seq[i: i+k] for i in range(len(seq) - k + 1)])

#modify to correct the count#
def diff_freqs(f1: Counter, f2: Counter) -> int:
    # f1 is always smaller
    c = 0
    
    for k in f1:
        d = f1[k] - f2[k] 
        if d > 0:
            c += d
    
    return c

def ani(s1, s2, k):
    
    s1, s2 = (s1, s2) if len(s1) < len(s2) else (s2, s1)
    f1 = kfreqs(s1, k)
    f2 = kfreqs(s2, k)
    delta = diff_freqs(f1, f2)
    return  delta / k / len(s1)


#update to get the ani matrix
def pair_ani(seq_coll:dict, pair_distance, k) -> numpy.ndarray: 
    """
    Get a pairwise ani distance matrix. 
    """
    seq_names = pair_distance.names
    num_seqs = len(seq_names)
    anis = numpy.zeros((num_seqs, num_seqs), dtype=float)

    #calculate ani matrix
    for x in range(num_seqs-1):
        name1 = seq_names[x]
        for y in range(x + 1, num_seqs):
            name2 = seq_names[y]
            dist = ani(seq_coll[name1], seq_coll[name2], k) 
            anis[x,y] = dist
            anis[y,x] = dist
            

    return anis


#tree construction
def nj_Kimura_corr_frac(distance_matrix): 
    """
    should return a PhyloNode class, which constructed by corrected fractional difference.
    """
    names = distance_matrix.names
    num_seqs = distance_matrix.shape[0]
    indices = numpy.tril_indices(num_seqs)

    array_distance = distance_matrix.to_array()
    dist_frac_diff = {}
    for i in indices[0]:
        for j in indices[1]:
            frac_diff = array_distance[i][j] 
            #Kimura correction
            dist_frac_diff[(names[i], names[j])] =  -log(1- frac_diff - 0.2*frac_diff**2)
    return nj.nj(dist_frac_diff, show_progress=False)

def nj_Kimura_corr_ani(distance_matrix, ani_matrix, parameters): 
    """
    should return a PhyloNode class, which constructed by corrected fractional difference estimates.
    """
    names = distance_matrix.names
    num_seqs = distance_matrix.shape[0]
    indices = numpy.tril_indices(num_seqs)

    array_distance = distance_matrix.to_array()
    dist_estimates = {}
    c = parameters[3]
    for i in indices[0]:
        for j in indices[1]:
            ani = ani_matrix[i][j] 
            #predict frac_diff
            parameters[3] = c - ani
            if abs(ani - cubic_function(min(numpy.real(numpy.roots(parameters))), 
                                  6.96178476e-01, -1.36789855e+00,  9.49234106e-01,  8.45293002e-04)) < 1e-9 :
               estimate_D = min(numpy.real(numpy.roots(parameters)))
            else:
                estimate_D = max(numpy.real(numpy.roots(parameters)))
            #Kimura correction
            dist_estimates[(names[i], names[j])] =  -log(1- estimate_D - 0.2*estimate_D**2)
    return nj.nj(dist_estimates, show_progress=False)


In [3]:
#load cogent3 test data
aln = load_aligned_seqs("~/repos/Cogent3/tests/data/brca1.fasta", moltype="dna")

## Fit an appropriate model to a.a.

In [4]:
aa_aln = aln.get_translation(incomplete_ok=True, gc=1)

In [5]:
#calculate fractional difference 
aa_pairwise_distance = aa_aln.distance_matrix()

In [6]:
aa_seq_coll = aa_aln.degap()
aa_data = aa_seq_coll.to_dict()

In [7]:
aa_anis_k_2 = pair_ani(aa_data, aa_pairwise_distance, k=2)
aa_anis_k_3 = pair_ani(aa_data, aa_pairwise_distance, k=3)
aa_anis_k_4 = pair_ani(aa_data, aa_pairwise_distance, k=4)
aa_anis_k_5 = pair_ani(aa_data, aa_pairwise_distance, k=5)
aa_anis_k_6 = pair_ani(aa_data, aa_pairwise_distance, k=6)

In [8]:
#get vectors of ani and pairwise distance for data with gaps
num_seqs = aa_pairwise_distance.shape[0]
indices = numpy.tril_indices(num_seqs)

aa_anis_k_2 = aa_anis_k_2[indices]
aa_anis_k_3 = aa_anis_k_3[indices]
aa_anis_k_4 = aa_anis_k_4[indices]
aa_anis_k_5 = aa_anis_k_5[indices]
aa_anis_k_6 = aa_anis_k_6[indices]

aa_frac_diff = aa_pairwise_distance.array[indices] 

In [9]:
fig = make_subplots(rows=5, cols=1,
x_title="fractional difference", y_title="ani")

fig.add_trace(go.Scatter(
    x=aa_frac_diff,
    y=aa_anis_k_2,
    mode='markers',
    name='k = 2'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=aa_frac_diff,
    y=aa_anis_k_3,
    mode='markers',
    name='k = 3'
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=aa_frac_diff,
    y=aa_anis_k_4, 
    mode='markers',
    name='k = 4'
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=aa_frac_diff,
    y=aa_anis_k_5, 
    mode='markers',
    name='k = 5'
), row=4, col=1)

fig.add_trace(go.Scatter(
    x=aa_frac_diff,
    y=aa_anis_k_6, 
    mode='markers',
    name='k = 6'
), row=5, col=1)

fig.update_layout(height=1000, width=1000, title_text="Comparison between ANI and fractional difference")
fig.show()

In [10]:
aa_frac_diff_ordered = numpy.array(sorted(aa_frac_diff)) #cast frac_identical into list to order the data

In [11]:
#fit data with models to get parameters, for k = 4
aa_popt_cub = curve_fit(cubic_function, aa_frac_diff, aa_anis_k_4)[0]
aa_popt_cub

array([ 6.96178476e-01, -1.36789855e+00,  9.49234106e-01,  8.45293002e-04])

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=aa_frac_diff, y=aa_anis_k_4, mode='markers', name='emprical data'))
fig.add_trace(go.Scatter(x=aa_frac_diff_ordered, y=cubic_function(aa_frac_diff_ordered, *aa_popt_cub), mode='lines', name='fit line'))
fig.add_annotation(x=0.48, y=0.17,
            text="y = 0.696x^3 - 1.368x^2 + 0.949x + 0.001",
            showarrow = False)
fig.update_layout(title='Polynomial models for a.a.',
                   xaxis_title='fractional difference',
                   yaxis_title='ani')
fig.show()

## Use brca1 data to construct the tree by ```frac_diff``` estimates

In [13]:
#after correcting to Kimura protein distance, construct fraction difference tree
tree_frac_diff = nj_Kimura_corr_frac(aa_pairwise_distance)

fig_frac_diff = tree_frac_diff.get_figure(width=800, height=800)
fig_frac_diff.scale_bar = "top right"
fig_frac_diff.show()

In [14]:
#after predict fraction difference by ani, construct tree
aa_anis_k_4 = pair_ani(aa_data, aa_pairwise_distance, k=4)
tree_ani = nj_Kimura_corr_ani(aa_pairwise_distance, aa_anis_k_4, parameters=aa_popt_cub)

fig_ani = tree_ani.get_figure(width=800, height=800)
fig_ani.scale_bar = "top right"
fig_ani.show()

## Use more emprical data to check fraction identity estimate

### Case 1 : SCA1-cds data from cogent3 

In [15]:
#load up
reader = io.load_unaligned(format="fasta")
to_aa = translate.translate_seqs()
process = reader + to_aa
seqs_sca1 = process("~/repos/Cogent3/doc/data/SCA1-cds.fasta")

aa_aligner = progressive_align("protein")
aligned = aa_aligner(seqs_sca1)
aligned

,0
Human,MKSNQERSNECLPPKKREIPATSRSSEEKAPTLPSDNHRVEGTAWLPGNPGGRGHGGGRH
Chimp,............................................................
Mouse Lemur,...............................A.......A..AP................
Rat,........................P.....TA......C...V....ST..S........
Mouse,........................P.....TA......C...V....ST..I........
Macaque,........................P......A............................


In [16]:
#compute fractional difference
pairwise_distance_sca1 = aligned.distance_matrix()
pairwise_distance_sca1

Chimp,0.0000,0.0050,0.0112,0.1093,0.0799,0.1056
Human,0.0050,0.0000,0.0075,0.1052,0.0759,0.1015
Macaque,0.0112,0.0075,0.0000,0.1027,0.0772,0.0990
Mouse,0.1093,0.1052,0.1027,0.0000,0.1090,0.0203
Mouse Lemur,0.0799,0.0759,0.0772,0.1090,0.0000,0.1028
Rat,0.1056,0.1015,0.0990,0.0203,0.1028,0.0000


In [17]:
#compute ani
aa_seq_coll_sca1 = aligned.degap()
data_sca1 = aa_seq_coll_sca1.to_dict()
aa_anis_k_4_sca1 = pair_ani(data_sca1, pairwise_distance_sca1, k=4)

In [18]:
#after correcting to Kimura distance, construct fraction difference tree
tree_frac_diff_sca1 = nj_Kimura_corr_frac(pairwise_distance_sca1)

fig_frac_diff_sca1 = tree_frac_diff_sca1.get_figure(width=500, height=500)
fig_frac_diff_sca1.scale_bar = "top right"
fig_frac_diff_sca1.show()

In [19]:
#after predict fraction identity by ani, construct tree
tree_ani_sca1 = nj_Kimura_corr_ani(pairwise_distance_sca1, aa_anis_k_4_sca1, parameters=aa_popt_cub)

fig_ani_sca1 = tree_ani_sca1.get_figure(width=500, height=500)
fig_ani_sca1.scale_bar = "top right"
fig_ani_sca1.show()

In [20]:
#do MSA using ani tree as a guide
aligner = progressive_align("protein", guide_tree=tree_ani_sca1)
aligned = aligner(seqs_sca1)
aligned

,0
Human,MKSNQERSNECLPPKKREIPATSRSSEEKAPTLPSDNHRVEGTAWLPGNPGGRGHGGGRH
Mouse Lemur,...............................A.......A..AP................
Rat,........................P.....TA......C...V....ST..S........
Mouse,........................P.....TA......C...V....ST..I........
Macaque,........................P......A............................
Chimp,............................................................


### Case 2 : Transcriptome data of turtle, bird, crocodiles from Chiari et al. 2012

In [21]:
#load data
aln_turtle = load_aligned_seqs("~/Documents/lab/alignment-free/data/turtle.fasta", moltype="dna")

#get translated
aa_aln_turtle = aln_turtle.get_translation(incomplete_ok=True, gc=1) 

In [22]:
#compute fractional difference
aa_pair_distance_turtle = aa_aln_turtle.distance_matrix()
aa_pair_distance_turtle

Anolis,0.0000,0.1157,0.1461,0.1507,0.1504,0.1157,0.1834,0.0979,0.1204,0.1004,0.0763,0.1180,0.1212,0.0363,0.1587,0.0666
Gallus,0.1157,0.0000,0.1286,0.1387,0.1341,0.0434,0.1779,0.0591,0.0873,0.0847,0.0664,0.0863,0.0875,0.0787,0.1452,0.0990
Homo,0.1461,0.1286,0.0000,0.0981,0.1127,0.1249,0.1893,0.0953,0.1462,0.1225,0.0916,0.1248,0.1252,0.1284,0.1501,0.1172
Monodelphis,0.1507,0.1387,0.0981,0.0000,0.1185,0.1377,0.1931,0.0998,0.1483,0.1295,0.0856,0.1298,0.1384,0.1310,0.1578,0.1321
Ornithorhynchus,0.1504,0.1341,0.1127,0.1185,0.0000,0.1337,0.1961,0.1043,0.1571,0.1141,0.0841,0.1267,0.1285,0.1199,0.1584,0.1097
Taeniopygia,0.1157,0.0434,0.1249,0.1377,0.1337,0.0000,0.1790,0.0698,0.0852,0.0849,0.0665,0.0941,0.0931,0.0748,0.1431,0.0983
Xenopus,0.1834,0.1779,0.1893,0.1931,0.1961,0.1790,0.0000,0.1565,0.1914,0.1621,0.1400,0.1828,0.1930,0.1488,0.1530,0.1629
alligator,0.0979,0.0591,0.0953,0.0998,0.1043,0.0698,0.1565,0.0000,0.0062,0.0486,0.0434,0.0707,0.0578,0.0349,0.1071,0.0902
caiman,0.1204,0.0873,0.1462,0.1483,0.1571,0.0852,0.1914,0.0062,0.0000,0.0833,0.0506,0.0689,0.0677,0.1003,0.1450,0.1026
caretta,0.1004,0.0847,0.1225,0.1295,0.1141,0.0849,0.1621,0.0486,0.0833,0.0000,0.0094,0.0076,0.0270,0.0693,0.1165,0.0790
chelonoidis_nigra,0.0763,0.0664,0.0916,0.0856,0.0841,0.0665,0.1400,0.0434,0.0506,0.0094,0.0000,0.0097,0.0304,0.0541,0.1054,0.0723


In [23]:
#compute ani 
aa_seq_coll_turtle = aa_aln_turtle.degap()
aa_data_turtle = aa_seq_coll_turtle.to_dict()

aa_anis_k_4_turtle = pair_ani(aa_data_turtle, aa_pair_distance_turtle, k=4)

In [24]:
#after correcting to Kimura distance, construct fraction difference tree
tree_frac_diff_turtle = nj_Kimura_corr_frac(aa_pair_distance_turtle)

fig_frac_diff_turtle = tree_frac_diff_turtle.get_figure(width=700, height=700)
fig_frac_diff_turtle.scale_bar = "top right"
fig_frac_diff_turtle.show()

In [25]:
#after predict fraction identity by ani, construct tree
tree_ani_turtle = nj_Kimura_corr_ani(aa_pair_distance_turtle, aa_anis_k_4_turtle, parameters=aa_popt_cub)

fig_ani_turtle = tree_ani_turtle.get_figure(width=700, height=700)
fig_ani_turtle.scale_bar = "top right"
fig_ani_turtle.show()